# HTTP 動態網頁架構說明


* 了解動態網頁的資料爬蟲策略
* 知道非同步網頁載入機制（Ajax）
* 學習兩種對應動態網頁爬蟲的的策略


## 作業目標

回答以下問題：

1. 動態網頁跟靜態網頁的差別是什麼？原本靜態網頁的做法會產生什麼問題或是缺點？
2. 還記得我們在 ETTODAY 靜態爬蟲實作練習中，有請大家完成一個題目「取出今天所有的發文」，但仔細看其實並沒有真的把當天所有的新聞都抓回來， 試著回答看看原因是什麼？及該如何檢查？


### 1. 動態網頁跟靜態網頁的差別是什麼？原本靜態網頁的做法會產生什麼問題或是缺點？

In [ ]:
1. 在靜態網頁中，每當有資料變動時，網頁都必須全部更新；動態網頁則只須更新與資料有關的部份。
2. 在靜態網頁中，無法以parsing標籤的方式獲取我們需要的資料。

### 2. 還記得我們在 ETTODAY 靜態爬蟲實作練習中，有請大家完成一個題目「取出今天所有的發文」，但仔細看其實並沒有真的把當天所有的新聞都抓回來， 試著回答看看原因是什麼？及該如何檢查？

In [64]:
import requests
from bs4 import BeautifulSoup

req = requests.get('https://www.ettoday.net/news/news-list.htm')
response = BeautifulSoup(req.text)
cat_list = []
for li in response.find('div', class_='part_menu_2').find_all('li'):
    if li.text != '|':
        cat_list.append(li.text)
# 模擬封包的標頭
headers = {'authority': 'www.ettoday.net',
           'method': 'POST',
           'path': '/show_roll.php',
           'scheme': 'https',
           'accept': '*/*',
           'accept-encoding': 'gzip, deflate, br',
           'content-length': '56',
           'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
           'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
           'cookie': 'et_token=98bd01d2e66ec20e339a4d64022894c4; check_pc_mobile=pc; _ga=GA1.2.794349579.1576233576; __auc=062d765616efed7a976cd076043; __gads=ID=3e00fef8be456835:T=1576233577:S=ALNI_MZMhblsyeICGMY96Hk9vIyM2ZiNjw; dable_uid=17320488.1576127568145; oid=%257B%2522oid%2522%253A%252202484fff-1c9e-11ea-9970-0242ac120003%2522%252C%2522ts%2522%253A1576127555%252C%2522v%2522%253A%25221.0%2522%257D; _huid=455ecc0b-4f32-4611-be64-13589af4104f; adid=455ecc0b-4f32-4611-be64-13589af4104f; webpush=1; et_tag=nMAF; dcs_local_cid=cyeg7k96n1; truvid_protected={"fraud":false,"level":2,"geo":"TW"}; et_client_country=TW; crazyAdToday_1=9; _gid=GA1.2.1818069892.1576823538; __asc=a09ea3cc16f2201c34a7a743842; crazyAdToday_4=0; adC02-63117=1; adC02-63117-expire=Fri%20Dec%2020%202019%2016%3A32%3A19%20GMT%2B0800%20(Taipei%20Standard%20Time); crazyAdToday_6=2; _ht_hi=1; _ht_em=1; GED_PLAYLIST_ACTIVITY=W3sidSI6InRnZ3ciLCJ0c2wiOjE1NzY4MjM2NTksIm52IjoxLCJ1cHQiOjE1NzY4MjM1MzYsImx0IjoxNTc2ODIzNjU5fV0.',
           'origin': 'https://www.ettoday.net',
           'referer': 'https://www.ettoday.net/news/news-list.htm',
           'sec-fetch-mode': 'cors',
           'sec-fetch-site': 'same-origin',
           'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) snap Chromium/79.0.3945.79 Chrome/79.0.3945.79 Safari/537.36',
           'x-requested-with': 'XMLHttpRequest'}
#
# 模擬表格欄位資料，循環送出 offset(>1)，即可獲得所有發文
#
data = {'offset': "3",
        'tPage': "3",
        'tFile': "20191220.xml",
        'tOt': "0",
        'tSi': "100",
        'tAr': "0"}

req = requests.post('https://www.ettoday.net/show_roll.php', headers=headers, data=data)
response = BeautifulSoup(req.text)
# Create a empty dictionary to save news entries.
news_dict = {}
for i in  range(1, len(cat_list)):
    news_dict[cat_list[i]] = []
# Insert each news entry to the dictionary.
for h3 in response.find_all('h3'):
    news = dict(n_time=h3.span.text, n_desc=h3.a.text, n_link='https://www.ettoday.net' + h3.a['href'])
    news_dict[h3.em.text].append(news)
news_dict

{'政治': [{'n_time': '2019/12/20 14:17',
   'n_desc': '台中第一所原住民實驗中學課程揭牌\u3000盧秀燕：總預算4成花在教育',
   'n_link': 'https://www.ettoday.net/news/20191220/1606538.htm'},
  {'n_time': '2019/12/20 14:06',
   'n_desc': '大選激化！雲林民進黨議員收詛咒信\u3000「你肺癌、你全家都肺癌」',
   'n_link': 'https://www.ettoday.net/news/20191220/1606493.htm'}],
 '財經': [],
 '論壇': [],
 '國際': [],
 '大陸': [],
 '社會': [],
 '地方': [],
 '新奇': [],
 '生活': [],
 '寵物動物': [{'n_time': '2019/12/20 14:13',
   'n_desc': '鏟屎官救星！19KG高效除臭礦砂在此\u3000多貓核彈「屎炸」也不怕\u3000美式賣場折價110元',
   'n_link': 'https://www.ettoday.net/news/20191220/1601474.htm'}],
 '影劇': [{'n_time': '2019/12/20 14:15',
   'n_desc': '廖科溢錄外景撞見婚禮\u3000秒變「婚禮攝影」沾喜氣',
   'n_link': 'https://www.ettoday.net/news/20191220/1606469.htm'},
  {'n_time': '2019/12/20 14:14',
   'n_desc': '《葉問4》限定場賣破300萬\u3000甄子丹最後一次「打木人樁」逼哭鐵粉',
   'n_link': 'https://www.ettoday.net/news/20191220/1606559.htm'}],
 '體育': [{'n_time': '2019/12/20 14:12',
   'n_desc': 'ABL／林志傑受傷空缺誰來頂？\u3000富邦勇士還沒有決定',
   'n_link': 'https://www.ettoday.